<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">The Plays of William Shakespeare, XML and SingleStoreDB</h1>
    </div>
</div>

In [2]:
!pip cache purge --quiet

In [5]:
!pip install lxml --quiet

In [6]:
import pandas as pd

from lxml import etree

In [7]:
shared_tier_check = %sql SHOW VARIABLES LIKE "is_shared_tier"
if not shared_tier_check or shared_tier_check[0][1] == "OFF":
    %sql DROP DATABASE IF EXISTS xml_db;
    %sql CREATE DATABASE IF NOT EXISTS xml_db;

1 rows affected.

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SQLAlchemy to make connections to the selected database.</p>
    </div>
</div>

In [8]:
from sqlalchemy import *

db_connection = create_engine(connection_url)

In [9]:
%%sql
DROP TABLE IF EXISTS plays;

CREATE TABLE IF NOT EXISTS plays (
    id INT PRIMARY KEY NOT NULL,
    play LONGTEXT NOT NULL
);

++
||
++
++

In [10]:
%%sql
CREATE PIPELINE plays
AS LOAD DATA STAGE 'all_plays.csv'
SKIP DUPLICATE KEY ERRORS
INTO TABLE plays
FORMAT CSV
COLUMNS TERMINATED BY '|'
(id,play);

++
||
++
++

In [11]:
%%sql
START PIPELINE plays;

++
||
++
++

In [12]:
%%sql
SELECT id,
    SUBSTR(play, LOCATE('<TITLE>', play) + LENGTH('<TITLE>'),
    LOCATE('</TITLE>', play) - LOCATE('<TITLE>', play) - LENGTH('<TITLE>')) AS title
FROM plays
ORDER BY id;

37 rows affected.

id,title
1,The Tragedy of Antony and Cleopatra
2,All's Well That Ends Well
3,As You Like It
4,The Comedy of Errors
5,The Tragedy of Coriolanus
6,Cymbeline
7,A Midsummer Night's Dream
8,"The Tragedy of Hamlet, Prince of Denmark"
9,The First Part of Henry the Fourth
10,The Second Part of Henry the Fourth


In [13]:
result = %sql SELECT play FROM plays WHERE play LIKE '%Romeo and Juliet%'

play = " ".join([row.play for row in result])

1 rows affected.

In [14]:
root = etree.fromstring(play)

In [16]:
xpath_q1 = "/PLAY/TITLE"
play_title = root.xpath(xpath_q1)[0]
print(play_title.text)

The Tragedy of Romeo and Juliet


In [17]:
xpath_q2 = "/PLAY/PERSONAE/PERSONA"
personae = root.xpath(xpath_q2)
for persona in personae:
    print(persona.text)

ESCALUS, prince of Verona. 
PARIS, a young nobleman, kinsman to the prince.
An old man, cousin to Capulet. 
ROMEO, son to Montague.
MERCUTIO, kinsman to the prince, and friend to Romeo.
BENVOLIO, nephew to Montague, and friend to Romeo.
TYBALT, nephew to Lady Capulet.
BALTHASAR, servant to Romeo.
PETER, servant to Juliet's nurse.
ABRAHAM, servant to Montague.
An Apothecary. 
Three Musicians.
Page to Paris; another Page; an officer.
LADY MONTAGUE, wife to Montague.
LADY CAPULET, wife to Capulet.
JULIET, daughter to Capulet.
Nurse to Juliet. 
Citizens of Verona; several Men and Women, relations to both houses; Maskers, Guards, Watchmen, and Attendants.
Chorus.


In [18]:
xpath_q3 = "count(//SPEECH[SPEAKER='JULIET'])"
juliet_count = root.xpath(xpath_q3)
print(juliet_count)

118.0


In [19]:
xpath_q4 = "count(//SPEECH[SPEAKER='JULIET' and preceding-sibling::SPEECH[1][SPEAKER='ROMEO']])"
romeo_and_juliet_count = root.xpath(xpath_q4)
print(romeo_and_juliet_count)

37.0


## Cleanup

In [20]:
%%sql
STOP PIPELINE plays;

++
||
++
++

In [21]:
%%sql
DROP PIPELINE plays;

++
||
++
++

In [22]:
%%sql
DROP TABLE IF EXISTS plays;

++
||
++
++

In [23]:
shared_tier_check = %sql SHOW VARIABLES LIKE "is_shared_tier"
if not shared_tier_check or shared_tier_check[0][1] == "OFF":
    %sql DROP DATABASE IF EXISTS xml_db;

1 rows affected.